<a href="https://colab.research.google.com/github/dAn-solution/competition/blob/main/Prob_kiva_012.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kiva／クラウドファンディングの資金調達額予測
- Ridge回帰 を実施

### Google Driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'

Mounted at /content/drive
/content/drive/My Drive


### ライブラリのインストール、インポート

In [ ]:
# カレントディレクトリを変更
import os
os.chdir('/content/drive/My Drive/Probdata/kiva/')
print(os.getcwd())

/content/drive/My Drive/Probdata/kiva


In [ ]:
class Config():
    root_path = './'
    input_path = os.path.join(root_path, 'input')
    output_path = os.path.join(root_path, 'output')
    result_path = os.path.join(root_path, 'result')
    bert_model_name = 'bert-base-uncased'
    seed = 42
    debug = False

In [ ]:
import pandas as pd
import numpy as np
import datetime
import warnings

import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge

### データの読み込み

In [ ]:
train_df = pd.read_csv(f'{Config.input_path}/train.csv')
test_df = pd.read_csv(f'{Config.input_path}/test.csv')
sample_df = pd.read_csv(f'{Config.input_path}/sample_submission.csv')

train_tags_pca = np.load(f'{Config.result_path}/train_tags_pca.npy')
test_tags_pca = np.load(f'{Config.result_path}/test_tags_pca.npy')
train_bert_pca = np.load(f'{Config.result_path}/train_bert_pca.npy')
test_bert_pca = np.load(f'{Config.result_path}/test_bert_pca.npy')
train_bert_018_pca = np.load(f'{Config.result_path}/train_bert_018_pca.npy')
test_bert_018_pca = np.load(f'{Config.result_path}/test_bert_018_pca.npy')

train_X_pca = np.load(f'{Config.result_path}/train_X_pca.npy')
test_X_pca = np.load(f'{Config.result_path}/test_X_pca.npy')

## Ridge回帰の実行

- tag項目の設定

In [ ]:
# train_tags_df = train_df['TAGS'].str.get_dummies(sep=',').add_prefix('tag_')
# test_tags_df = test_df['TAGS'].str.get_dummies(sep=',').add_prefix('tag_')

In [ ]:
# # 各々に無い項目を追加
# lack_col = list(set(train_tags_df.columns) - set(test_tags_df.columns))
# if len(lack_col) != 0:
#     for col in lack_col:
#         test_tags_df[col] = [0]*len(test_tags_df)

# lack_col = list(set(test_tags_df.columns) - set(train_tags_df.columns))
# if len(lack_col) != 0:
#     for col in lack_col:
#         train_tags_df[col] = [0]*len(train_tags_df)

In [ ]:
# # trainデータを目的変数と説明変数に分割
# train_columns = ["ORIGINAL_LANGUAGE", "ACTIVITY_NAME", "SECTOR_NAME", "COUNTRY_CODE","CURRENCY_POLICY",
#        "CURRENCY","REPAYMENT_INTERVAL", "DISTRIBUTION_MODEL"]
target_column = "LOAN_AMOUNT"

y = train_df[target_column]
# X = train_df[train_columns]
# test_X = test_df[train_columns]

In [ ]:
# # 説明変数をラベルエンコーディング
# warnings.simplefilter('ignore')
# for column in X.columns:
#   le = preprocessing.LabelEncoder()
#   target_column = X[column]
#   test_target_column = test_X[column]  
#   le.fit(target_column)
#   label_encoded_column = le.transform(target_column)
#   test_label_encoded_column = le.transform(test_target_column)
#   X[column] = pd.Series(label_encoded_column).astype("category")
#   test_X[column] = pd.Series(test_label_encoded_column).astype("category")

In [ ]:
train_pca = np.concatenate([train_X_pca, train_tags_pca, train_bert_pca, train_bert_018_pca], axis=1)
test_pca = np.concatenate([test_X_pca, test_tags_pca, test_bert_pca, test_bert_018_pca], axis=1)

# X = pd.concat([X, pd.DataFrame(train_pca)], axis=1)
# test_X = pd.concat([test_X, pd.DataFrame(test_pca)], axis=1)
X = pd.DataFrame(train_pca)
test_X = pd.DataFrame(test_pca)
print(X.shape)
print(test_X.shape)

(91333, 18)
(91822, 18)


In [ ]:
# trainデータを訓練用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = Config.seed)

In [ ]:
# 学習/予測

y_oof = np.zeros(len(y_train))
meta_preds = []
y_preds = []
kf = StratifiedKFold(n_splits=5)
for fold, (tr_idx, vl_idx) in enumerate(kf.split(X_train, y_train)):
    x_tr_fold = X.iloc[tr_idx]
    y_tr_fold = y.iloc[tr_idx]
    x_vl_fold = X.iloc[vl_idx]
    y_vl_fold = y.iloc[vl_idx]

    model = Ridge(alpha=10)
    model.fit(x_tr_fold, y_tr_fold)
    y_oof[vl_idx] = model.predict(x_vl_fold)
    meta_preds.append(model.predict(X))
    y_preds.append(model.predict(test_X))
    print(
        f'fold {fold} score:', mean_absolute_error(y_oof[vl_idx], y_vl_fold)
    )

print('X score:', mean_absolute_error(model.predict(X), y))

fold 0 score: 511.96836567627173


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


fold 1 score: 504.4035981509968
fold 2 score: 485.6940455655227
fold 3 score: 498.14461165216807
fold 4 score: 509.95950823814474
X score: 502.4110591460934


In [ ]:
# メタモデルへの引き渡し
train_012_df = pd.DataFrame(np.mean(meta_preds, axis=0))
train_012_df.to_csv(os.path.join(Config.result_path, "train_012.csv"), index=False)


test_012_df = pd.DataFrame(np.mean(y_preds, axis=0))
test_012_df.to_csv(os.path.join(Config.result_path, "test_012.csv"), index=False)

### 提出データの作成

In [ ]:
sample_df["LOAN_AMOUNT"] = np.mean(y_preds, axis=0)
print(sample_df)

       LOAN_ID  LOAN_AMOUNT
0      2041445   289.947602
1      1944435  1074.886068
2      2083354   800.011788
3      1993565   729.510040
4      2064272    98.600533
...        ...          ...
91817  1993862   659.030689
91818  2015070   315.101213
91819  1950349   495.019857
91820  1921580  1386.014888
91821  1976733   903.872732

[91822 rows x 2 columns]


In [ ]:
# submitファイルのファイル名に利用する作成時刻の取得

now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=1)))
date_time = "{0:%Y%m%d_%H%M}".format(now)

In [ ]:
# submitファイルの出力(Google Driveに出力）
# 形式：submit_yyyymmdd_hhmm.csv

sample_df.to_csv(os.path.join(Config.output_path, f"submit_{date_time}.csv"), index=False)
